In [1]:
import pandas as pd
import os

In [2]:
current_path = os.getcwd()
print(current_path)

c:\Users\sepujas\Dev\mat\notebooks


In [3]:
base_path = os.path.abspath(os.path.join(current_path, os.pardir))
print(base_path)

c:\Users\sepujas\Dev\mat


In [4]:
me2m_file_path = os.path.join(base_path, 'data', 'raw', 'tbl_me2m.txt')
print(me2m_file_path)

c:\Users\sepujas\Dev\mat\data\raw\tbl_me2m.txt


In [5]:
me2m_preprocessed_path = os.path.join(base_path, 'data', 'preprocessed', 'tbl_me2m.csv')
print(me2m_preprocessed_path)

c:\Users\sepujas\Dev\mat\data\preprocessed\tbl_me2m.csv


In [6]:
df_me2m = pd.read_csv(me2m_file_path, sep='\t', skiprows=3, encoding='latin1')

In [7]:
df_me2m.head()

,Unnamed: 0,POrg,Plnt,SLoc,Material,Type,Doc. Date,Supplier,Pur. Doc.,Quantity,...,Quantity.1,SKU,PGr,Net Price,Crcy,Per,To be del.,To be del.,To be inv.,To be inv.
0,NaN,8300,8330,1.0,2203491.0,NB,13.09.2024,9999 HILTI,4537359685,3.000,...,3.000,M,L11,"48,84",USD,1.0,3.000,"146.520,00",0,"0,00"
1,NaN,8300,8330,1.0,2203546.0,NB,13.09.2024,9999 HILTI,4537359687,25.250,...,25.250,PC,L11,"31,47",USD,100.0,25.250,"7.946,18",0,"0,00"
2,NaN,8300,8330,1.0,2208751.0,NB,13.09.2024,9999 HILTI,4537359689,62.350,...,62.350,PC,L11,"7,34",USD,10.0,62.350,"45.764,90",0,"0,00"
3,NaN,8300,8330,1.0,2238428.0,NB,13.09.2024,9999 HILTI,4537359691,1.500,...,1.500,PC,L11,"33,71",USD,1.0,1.500,"50.565,00",0,"0,00"
4,NaN,8300,8330,1.0,2203491.0,NB,13.09.2024,9999 HILTI,4537359693,690,...,690,M,L11,"48,84",USD,1.0,690,"33.699,60",0,"0,00"


In [8]:
unnamed_columns = [col for col in df_me2m.columns if 'Unnamed:' in col]
df_me2m.drop(columns=unnamed_columns, inplace=True)
column_titles = df_me2m.columns.tolist()
new_column_titles = {col: col.strip().replace(' ', '_').replace('-','_').replace('.','') for col in column_titles}
df_me2m.rename(columns=new_column_titles, inplace=True)

In [9]:
column_counts = df_me2m.columns.value_counts()
duplicate_columns = column_counts[column_counts > 1].index

In [10]:
for col in duplicate_columns:
    col_indices = [i for i, x in enumerate(df_me2m.columns) if x == col]
    for j, index in enumerate(col_indices):
        df_me2m.columns.values[index] = f"{col}_{j+1}"

In [11]:
df_me2m.columns.tolist()

['POrg',
 'Plnt',
 'SLoc',
 'Material',
 'Type',
 'Doc_Date',
 'Supplier',
 'Pur_Doc',
 'Quantity',
 'OUn',
 'Quantity1',
 'SKU',
 'PGr',
 'Net_Price',
 'Crcy',
 'Per',
 'To_be_del_1',
 'To_be_del_2',
 'To_be_inv_1',
 'To_be_inv_2']

In [12]:
df_me2m.dtypes

POrg             int64
Plnt             int64
SLoc           float64
Material       float64
Type            object
Doc_Date        object
Supplier        object
Pur_Doc          int64
Quantity        object
OUn             object
Quantity1       object
SKU             object
PGr             object
Net_Price       object
Crcy            object
Per            float64
To_be_del_1     object
To_be_del_2     object
To_be_inv_1     object
To_be_inv_2     object
dtype: object

In [13]:
def transform_columns(df):
    str_columns = [
        'Type', 'Supplier', 'OUn', 'SKU', 'PGr', 'Crcy']
    for col in str_columns:
        df[col] = df[col].astype(str).str.strip()

    numeric_columns = ['POrg', 'Plnt', 'SLoc', 'Material', 'Pur_Doc']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int).astype(str)

    df['Quantity'] = df['Quantity'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['Quantity1'] = df['Quantity1'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['Net_Price'] = df['Net_Price'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['To_be_del_1'] = df['To_be_del_1'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['To_be_del_2'] = df['To_be_del_2'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['To_be_inv_1'] = df['To_be_inv_1'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
    df['To_be_inv_2'] = df['To_be_inv_2'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)

    date_columns = ['Doc_Date']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", format='%d.%m.%Y')

transform_columns(df_me2m)

In [14]:
df_me2m.head()

,POrg,Plnt,SLoc,Material,Type,Doc_Date,Supplier,Pur_Doc,Quantity,OUn,Quantity1,SKU,PGr,Net_Price,Crcy,Per,To_be_del_1,To_be_del_2,To_be_inv_1,To_be_inv_2
0,8300,8330,1,2203491,NB,2024-09-13,9999 HILTI,4537359685,3000.0,M,3000.0,M,L11,48.84,USD,1.0,3000.0,146520.00,0.0,0.0
1,8300,8330,1,2203546,NB,2024-09-13,9999 HILTI,4537359687,25250.0,PC,25250.0,PC,L11,31.47,USD,100.0,25250.0,7946.18,0.0,0.0
2,8300,8330,1,2208751,NB,2024-09-13,9999 HILTI,4537359689,62350.0,PC,62350.0,PC,L11,7.34,USD,10.0,62350.0,45764.90,0.0,0.0
3,8300,8330,1,2238428,NB,2024-09-13,9999 HILTI,4537359691,1500.0,PC,1500.0,PC,L11,33.71,USD,1.0,1500.0,50565.00,0.0,0.0
4,8300,8330,1,2203491,NB,2024-09-13,9999 HILTI,4537359693,690.0,M,690.0,M,L11,48.84,USD,1.0,690.0,33699.60,0.0,0.0


In [15]:
df_me2m.dtypes

POrg                   object
Plnt                   object
SLoc                   object
Material               object
Type                   object
Doc_Date       datetime64[ns]
Supplier               object
Pur_Doc                object
Quantity              float64
OUn                    object
Quantity1             float64
SKU                    object
PGr                    object
Net_Price             float64
Crcy                   object
Per                   float64
To_be_del_1           float64
To_be_del_2           float64
To_be_inv_1           float64
To_be_inv_2           float64
dtype: object

In [16]:
df_me2m['key_plant'] = df_me2m['Plnt'] + '/' + df_me2m['Material']
df_me2m['key_plant'] = df_me2m['key_plant'].astype(str).str.strip()
df_me2m['key_material'] = df_me2m['POrg'] + '/' + df_me2m['Material']
df_me2m['key_material'] = df_me2m['key_material'].astype(str).str.strip()

In [17]:
df_me2m.to_csv(me2m_preprocessed_path, index=False, sep='|')

In [18]:
df_me2m.dtypes

POrg                    object
Plnt                    object
SLoc                    object
Material                object
Type                    object
Doc_Date        datetime64[ns]
Supplier                object
Pur_Doc                 object
Quantity               float64
OUn                     object
Quantity1              float64
SKU                     object
PGr                     object
Net_Price              float64
Crcy                    object
Per                    float64
To_be_del_1            float64
To_be_del_2            float64
To_be_inv_1            float64
To_be_inv_2            float64
key_plant               object
key_material            object
dtype: object